1. Initialisation

* Parameters set up 
* Initial states for $k_{0}$ and $a_{0}$ 
     
```python
    # Number of countries
    N = 2
    # Choose the simulation length for the solution procedure
    T = 2000
    # Utility-function parameter
    gam = 1
    # Capital share in output
    alpha = 0.36
    # Discount factor
    beta = 0.99
    # Depreciation rate 
    delta = 0.025
    # Persistence of the log of the productivity level
    rho = 0.95
    # Standard deviation of shocks to the log of the productivity level
    sigma = 0.01
    # Variance-covariance matrix
    vcv = sigma**(2)*(np.eye(N)+ np.ones(N))
    # Normalising constant
    A = (1-beta+beta*delta)/alpha/beta
    # welfare weight
    tau = 1
    # inital states
    k = np.ones((T+1,N))
    a = np.ones((1,N))
```

2. At iteration $p$ Use $b^{(p)}$ to simulate the model for $T$ periods
* Capital
\begin{align*}
k_{t+1}=\psi(k_{t},a_{t};b^{(p)})
\end{align*}

```python
    for i in range(T):
            x[i]= np.hstack((1,k[i], a[i]))
            k[i+1]= np.hstack((1,k[i], a[i]))@bk_1d
```
* Consumption
\begin{align*}
c_{t}=(1-\delta)k_{t}+ a_{t,j}f(k_{t})-k_{t+1,j}
\end{align*}

```python
    # Aggregated consumption
    C = (A*k0**alpha*a0 - k1+ (1-delta)*k0)@np.ones((N,1))

    # Individual consumption is the same for all countries
    c = C@np.ones((1,N))/N
```


3. Define $y_{t}$ to be the approximation of conditional expectation in equation (6) with given $J$ integration nodes $\{\epsilon_{t+1,j}\}_{j=1,...,J}$ and weights $\{\omega_{t+1,j}\}_{j=1,...,J}$ discussed in previous section:

\begin{align*}
y_{t}= \sum^{J}_{j=1}\{\omega_{t,j}\cdot(\beta\frac{u^{'}(c_{t+1,j})}{u^{'}(c_{t})}[1-\delta + a_{t+1,j}f^{'}(k_{t+1}) ]k_{t+1}) \} \tag{7} 
\end{align*}

```python
    for i in range(1, N+1):
        Y[0:T,i-1][:,np.newaxis]= Y[0:T,i-1][:,np.newaxis]+ weight_nodes[i-1, 0]*beta*c1[0:T, i-1][:,np.newaxis]**(-gam)/c[0:T,i-1][:,np.newaxis]**(-gam)*(1-delta+alpha*A*k1[0:T,i-1][:,np.newaxis]**(alpha-1)*a1[0:T,i-1][:,np.newaxis])*k1[0:T, i-1][:,np.newaxis]
```

Notice that $a_{t+1,j}$, $k_{t+2,j}$, and $c_{t+1,j}$ are calculated as following:
\begin{align*}
a_{t+1,j} = a^{\rho}_{t}exp(\epsilon_{t+1,j})
\end{align*}

```python
    a1 = a[0:T,:]**rho*np.exp(np.ones((T,1))@epsi_nodes[i-1,:][np.newaxis,])
```

\begin{align*}
k_{t+2,j} \equiv& \psi(k_{t+1}, a^{\rho}_{t}exp(\epsilon_{t+1,j});b^{(p)} ) \\
\end{align*}

```python
    k2 = Ord_Polynomial_N(np.hstack((k1, a1)),D)@bk_D
```

\begin{align*}
c_{t+1,j} = (1-\delta)k_{t+1}+ a_{t+1,j}f(k_{t+1})-k_{t+2,j}
\end{align*}

```python
    C1 = (A*k1**alpha*a1-k2+(1-delta)*k1)@np.ones((N,1))
    c1 = C1@np.ones((1,N))/N
```


4. Find the $\hat b$ that minimise the error term with chosen approximation method, depending on whether if it is the initial guess or updating the solution

```python
    bk_hat_D = Num_Stab_Approx(X[0:T-1,:], Y[0:T-1,:], RM, penalty, normalize)
```

5. Check convergence, jump out of stage one if satisfied
\begin{align*}
\frac{1}{T}\sum^{T}_{t=1}|\frac{k^{(p)}_{t+1}-k^{p-1}_{t+1}}{k^{(p)}_{t+1}}| < \varphi
\end{align*}

```python
    dif_GSSA_D = np.mean(abs(1-k/k_old))
    kdamp     = 0.1
    while dif_GSSA_D > 1e-4/10**D*kdamp:
```
Where the D inside the code denotes the polynomial degree and kdamp stands for damping parameter for iteration on the coefficients of the capital policy functions. The convergence parameter $\varphi$ depends on both values.

6. Compute $b^{p+1}$ for the next iteration
\begin{align*}
b^{(p+1)}=(1-\xi)b^{(p)} + \xi \hat b \tag{8} 
\end{align*}

```python
    bk_D = kdamp*bk_hat_D+ (1-kdamp)*bk_D
```

